### Group 5
Members:

Jaykumar Patel

Palak Bhootra

Alvin Henry

Saheer Shaikh

Leela Arumugapandian

Scanning and parsing news of the last two weeks from “newswire” website

In [54]:
# installing the module needed for scrapping nd to request the  url
!pip install beautifulSoup4
!pip install requests

In [55]:

from bs4 import BeautifulSoup #to scrape the site on basis of html tags
import urllib.request,sys,time #to reqquest the url
import datetime
import requests
from time import sleep #to have sometime in between 2 url requests

import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import yfinance as yf
from yahoofinancials import YahooFinancials

In [56]:
#Global constants

_URL = 'https://www.prnewswire.com/news-releases/news-releases-list/'
_HOMEURL = 'https://www.prnewswire.com/'
_MONTH_VALS = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun":6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}

In [57]:
count = 1
newUrl = ''
newsList = []

### Section 1

Scan (the last two weeks or the last week, upto you) and parse news from “newswire” websit.

In [58]:
def scrapeData(listing):
  '''Method to scrape the data ono the basis of hrml tags
   which calls the dateTime method to know till when the data is needed'''
  topics = listing.find('a')['href']
  # print('topic :' ,topics)
  link = _HOMEURL+topics
  extractNews = listing.h3
  date, time = getDateTime(listing.h3.small.text.split(","))
  # print('Date:',dateTime)
  heading = extractNews.text.split('\n')[2]
  # print(heading)
  return [heading,date,time,link]

In [59]:
class LimitReached(Exception):
    ''' 
        Custom Exception class to pause exception on reaching limit
    '''
    pass

In [60]:
def getDateTime(timeStamp):
    '''Helper function that return date and time of the news post'''
    
    if len(timeStamp) is not 1:
      year = int(timeStamp[1].strip())
      month = _MONTH_VALS[timeStamp[0].split(" ")[0]]
      date = int(timeStamp[0].split(" ")[1])
      # Check if the limit of 14 days has been reached or not
      if (datetime.datetime.today() - datetime.datetime(year, month, date)).days is 14:
        raise LimitReached
          # If yes raise exception to break loop
          # raise LimitReached
      date = datetime.datetime(year, month, date).strftime("%b %d, %Y")
      time = timeStamp[-1].split(" ")[-2]
    else:
      date = datetime.datetime.today().strftime("%b %d, %Y")
      time = timeStamp[0][0:-3]
    return date, time

This method is executed till the news of last 2 weeks is executed when the time is execeeded it throws LimitReached exception which helps the method to know when to stop

It also handles the exception when the page does not respond or throws an error

In [61]:
while True:
  newUrl = _URL + '?page=' + str(count) + '&pagesize=100'
  
  try:
    page=requests.get(newUrl)
  except Exception as e:  
    error_type, error_obj, error_info = sys.exc_info()  
    print ('ERROR FOR LINK:',_URL)  
    print (error_type, 'Line:', error_info.tb_lineno)
  
  soup = BeautifulSoup(page.text, "html.parser")
  news=soup.find_all('div',attrs={'class':'row arabiclistingcards'})

  try:
    for listing in news:
      newsList.append(scrapeData(listing))
  except LimitReached:
    break
  count += 1
  sleep(2)

In [62]:
newsList

[['Centurium Capital Releases Carbon Neutral Strategy and Roadmap',
  'Apr 22, 2022',
  '21:39 ',
  'https://www.prnewswire.com//news-releases/centurium-capital-releases-carbon-neutral-strategy-and-roadmap-301530715.html'],
 ['Circulating Fluidized Bed (CFB) Boilers Market to Advance at CAGR of 12.2% during 2021-2031; TMR Study',
  'Apr 22, 2022',
  '21:30 ',
  'https://www.prnewswire.com//news-releases/circulating-fluidized-bed-cfb-boilers-market-to-advance-at-cagr-of-12-2-during-20212031-tmr-study-301530310.html'],
 ['Los resultados iniciales del ensayo de la vacuna contra COVID-19-influenza de Novavax son los primeros en demostrar la viabilidad de la vacuna combinada',
  'Apr 22, 2022',
  '21:28 ',
  'https://www.prnewswire.com//news-releases/los-resultados-iniciales-del-ensayo-de-la-vacuna-contra-covid-19-influenza-de-novavax-son-los-primeros-en-demostrar-la-viabilidad-de-la-vacuna-combinada-897709885.html'],
 ['Terran Biosciences anuncia acordo de licenciamento com Sanofi para doi

With the help of csv module, we were able to save the dataframe to csv

In [63]:
header = ['News','Date','Time','Link']
with open('News.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(newsList)

In [64]:
df = pd.read_csv('News.csv')
df.head()

,News,Date,Time,Link
0,Centurium Capital Releases Carbon Neutral Stra...,"Apr 22, 2022",21:39,https://www.prnewswire.com//news-releases/cent...
1,Circulating Fluidized Bed (CFB) Boilers Market...,"Apr 22, 2022",21:30,https://www.prnewswire.com//news-releases/circ...
2,Los resultados iniciales del ensayo de la vacu...,"Apr 22, 2022",21:28,https://www.prnewswire.com//news-releases/los-...
3,Terran Biosciences anuncia acordo de licenciam...,"Apr 22, 2022",21:24,https://www.prnewswire.com//news-releases/terr...
4,Terran Biosciences anuncia un acuerdo de licen...,"Apr 22, 2022",21:21,https://www.prnewswire.com//news-releases/terr...


In [65]:
df.dropna()

,News,Date,Time,Link
0,Centurium Capital Releases Carbon Neutral Stra...,"Apr 22, 2022",21:39,https://www.prnewswire.com//news-releases/cent...
1,Circulating Fluidized Bed (CFB) Boilers Market...,"Apr 22, 2022",21:30,https://www.prnewswire.com//news-releases/circ...
2,Los resultados iniciales del ensayo de la vacu...,"Apr 22, 2022",21:28,https://www.prnewswire.com//news-releases/los-...
3,Terran Biosciences anuncia acordo de licenciam...,"Apr 22, 2022",21:24,https://www.prnewswire.com//news-releases/terr...
4,Terran Biosciences anuncia un acuerdo de licen...,"Apr 22, 2022",21:21,https://www.prnewswire.com//news-releases/terr...
...,...,...,...,...
9026,CLUB CAR TO ACQUIRE DANISH ELECTRIC VEHICLE MA...,"Apr 09, 2022",08:00,https://www.prnewswire.com//news-releases/club...
9027,Potassium Citrate Market Sourcing and Procurem...,"Apr 09, 2022",07:58,https://www.prnewswire.com//news-releases/pota...
9028,Graphite GTC Partners with Indian Institute of...,"Apr 09, 2022",07:07,https://www.prnewswire.com//news-releases/grap...
9029,Total Shade Inc Announces The Opening of The T...,"Apr 09, 2022",07:07,https://www.prnewswire.com//news-releases/tota...


To know which rows has news as null value, and extract such kind of news from dataframe

In [66]:
df[df.isnull().any(1)]

,News,Date,Time,Link
274,NaN,"Apr 22, 2022",13:00,https://www.prnewswire.com//news-releases/air-...
1204,NaN,"Apr 20, 2022",21:44,https://www.prnewswire.com//news-releases/rose...
1591,NaN,"Apr 20, 2022",11:28,https://www.prnewswire.com//news-releases/ther...
2648,NaN,"Apr 19, 2022",12:30,https://www.prnewswire.com//news-releases/bra-...
2741,NaN,"Apr 19, 2022",11:05,https://www.prnewswire.com//news-releases/free...
4081,NaN,"Apr 18, 2022",08:44,https://www.prnewswire.com//news-releases/beyo...
6230,NaN,"Apr 13, 2022",09:38,https://www.prnewswire.com//news-releases/cant...
6337,NaN,"Apr 13, 2022",09:00,https://www.prnewswire.com//news-releases/magi...
7029,NaN,"Apr 12, 2022",13:26,https://www.prnewswire.com//news-releases/pres...
7499,NaN,"Apr 12, 2022",09:00,https://www.prnewswire.com//news-releases/appl...


In [67]:
df = df[df['News'].notna()]

NewsList is being made to to have the news column to look at the defined stocks in that list

In [68]:
newsList = df['News']
newsList

0       Centurium Capital Releases Carbon Neutral Stra...
1       Circulating Fluidized Bed (CFB) Boilers Market...
2       Los resultados iniciales del ensayo de la vacu...
3       Terran Biosciences anuncia acordo de licenciam...
4       Terran Biosciences anuncia un acuerdo de licen...
                              ...                        
9026    CLUB CAR TO ACQUIRE DANISH ELECTRIC VEHICLE MA...
9027    Potassium Citrate Market Sourcing and Procurem...
9028    Graphite GTC Partners with Indian Institute of...
9029    Total Shade Inc Announces The Opening of The T...
9030    Residency & Citizenship Programs: Global Updat...
Name: News, Length: 9020, dtype: object

We have defined some stocks to look in the news, with the help of enumerate function we try to look in the news

In [69]:
_SYMBOLS = ['CDR','FHN','STYE','GDNSF','VOLT','TLMD']
stock_news = {}

for symb in _SYMBOLS:
    stock_news[symb] = [newsString for index, newsString in enumerate(newsList) if symb in newsString]

print(stock_news)

{'CDR': [], 'FHN': [], 'STYE': [], 'GDNSF': ['SHAREHOLDER ALERT: Weiss Law Reminds LHCG, NLSN, GDNSF, and VOLT Shareholders About Its Ongoing Investigations'], 'VOLT': ['SHAREHOLDER ALERT: Weiss Law Reminds LHCG, NLSN, GDNSF, and VOLT Shareholders About Its Ongoing Investigations', 'REVOLT Announces the Return of Highly Impactful Shows This Spring'], 'TLMD': []}
